# UE08: Datenaufbereitung und EDA am Beispiel Titanic

In dieser Übung werden wir den Titanic-Datensatz systematisch analysieren, um erste Erkenntnisse über die Überlebenschancen der Passagiere zu gewinnen. Dabei folgen wir einer bewährten Reihenfolge, um zuverlässige und aussagekräftige Ergebnisse zu erhalten. Zunächst laden wir die Daten und verschaffen uns einen Überblick über deren Struktur und Inhalt. Anschließend bereinigen wir den Datensatz, indem wir fehlende Werte, unplausible Einträge und potenzielle Ausreißer identifizieren und entsprechend behandeln. Erst nach dieser Bereinigung testen wir unsere Hypothesen, indem wir verschiedene Analysen und Visualisierungen durchführen. Diese strukturierte Vorgehensweise stellt sicher, dass unsere Analysen auf einer soliden Datenbasis beruhen und wir fundierte Aussagen treffen können.

## 0. Dataset im Überblick
- **PassengerId**: Eindeutige ID für jeden Passagier
- **Survived**: 1 = Überlebt, 0 = Nicht überlebt
- **Pclass**: Ticketklasse (1 = Erste Klasse, 2 = Zweite Klasse, 3 = Dritte Klasse)
- **Sex**: Geschlecht
- **Age**: Alter des Passagiers
- **SibSp**: Anzahl der Geschwister/Ehepartner an Bord
- **Parch**: Anzahl der Eltern/Kinder an Bord
- **Ticket**: Ticketnummer
- **Fare**: Ticketpreis
- **Cabin**: Kabinennummer
- **Embarked**: Hafen, an dem der Passagier eingestiegen ist (C = Cherbourg, Q = Queenstown, S = Southampton)

## 1. Vorgehensweise
Daten laden und Überblick verschaffen

    - Datensatz einlesen, erste Zeilen anschauen.
    - Infos zu Datentypen und fehlenden Werten prüfen.

Datenbereinigung

    - Fehlende Werte behandeln (z. B. durch Entfernen oder Imputieren).
    - Unplausible Werte und Ausreißer identifizieren.
    - Datentypen anpassen (z. B. Kategorien korrekt kodieren).

Hypothesen testen

    - Explorative Analyse durchführen (Univariate/Bivariate Analyse).
    - Die bereinigten Daten für Visualisierungen und statistische Berechnungen nutzen.
    
## 3. Hypothesen

1. Frauen hatten eine höhere Überlebensrate als Männer.
       
       - Begründung: Historisch gesehen wurden Frauen und Kinder bei der Evakuierung priorisiert („Women and children first“).
       - Überprüfung: Vergleich der Überlebensraten zwischen Männern und Frauen.

2. Passagiere der ersten Klasse hatten eine höhere Überlebenswahrscheinlichkeit als die der dritten Klasse.
        
       - Begründung: Personen in der ersten Klasse hatten besseren Zugang zu Rettungsbooten.
       - Überprüfung: Analyse der Überlebensraten nach Ticketklasse (Pclass).

3. Jüngere Passagiere hatten eine höhere Überlebenschance.

        - Begründung: Kinder wurden bevorzugt gerettet.
        - Überprüfung: Untersuchung der Altersverteilung zwischen Überlebenden und Nicht-Überlebenden.

4. Der Ticketpreis (Fare) könnte mit der Überlebensrate korrelieren.
    
        - Begründung: Höhere Ticketpreise sind mit der ersten Klasse verbunden, die eine höhere Überlebenswahrscheinlichkeit hatte.
        - Überprüfung: Korrelation zwischen Ticketpreis und Überlebensstatus.

5. Passagiere, die alleine reisten, hatten eine geringere Überlebenschance.
    
        - Begründung: Familien könnten sich gegenseitig unterstützt haben.
        - Überprüfung: Zusammenhang zwischen der Anzahl der Familienmitglieder an Bord (SibSp und Parch) und der Überlebensrate.
---
> Für die eine oder andere Aufgabe ist `groupby` überaus hilfreich: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

In [39]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
df = pd.read_csv("titanic_train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [7]:
df.notna().sum()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

In [29]:
# Frauen hatten eine höhere Überlebensrate als Männer.

df.groupby('Sex')['Survived'].mean()

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

In [11]:
# Passagiere der ersten Klasse hatten eine höhere Überlebenswahrscheinlichkeit als die der dritten Klasse.

df.groupby('Pclass')['Survived'].mean()

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

In [24]:
# Jüngere Passagiere hatten eine höhere Überlebenschance.

age_bins = pd.cut(df['Age'], bins=range(0, 85, 5))
df.groupby(age_bins, observed=True)['Survived'].agg(["mean", "count"])

,mean,count
Age,,
"(0, 5]",0.704545,44
"(5, 10]",0.350000,20
"(10, 15]",0.578947,19
"(15, 20]",0.343750,96
"(20, 25]",0.344262,122
"(25, 30]",0.388889,108
"(30, 35]",0.465909,88
"(35, 40]",0.417910,67
"(40, 45]",0.361702,47


In [34]:
# Der Ticketpreis (Fare) könnte mit der Überlebensrate korrelieren.

ticket_bins = pd.cut(df['Fare'], bins=range(0, 550, 40))
df.groupby(ticket_bins, observed=True)['Survived'].agg(["mean", "count"])

,mean,count
Fare,,
"(0, 40]",0.325714,700
"(40, 80]",0.549020,102
"(80, 120]",0.805556,36
"(120, 160]",0.750000,16
"(160, 200]",1.000000,2
"(200, 240]",0.666667,9
"(240, 280]",0.625000,8
"(480, 520]",1.000000,3


In [45]:
df[['Fare', 'Survived']].corr().iloc[0, 1]

np.float64(0.2573065223849626)

In [26]:
# Passagiere, die alleine reisten, hatten eine geringere Überlebenschance.

df['FamilySize'] = df['SibSp'] + df['Parch']
df.groupby('FamilySize')['Survived'].agg(["mean", "count"])

,mean,count
FamilySize,,
0,0.303538,537
1,0.552795,161
2,0.578431,102
3,0.724138,29
4,0.200000,15
5,0.136364,22
6,0.333333,12
7,0.000000,6
10,0.000000,7
